<a href="https://colab.research.google.com/github/erlichsefi/ScrapeAnything/blob/main/browser_base_translation%20/AutoJavaScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Converting sites to dataframe

# First: install selenium & chromium

In [1]:
# According to: https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
%%capture
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium
apt install chromium-chromedriver
pip install pandas
pip install openai

In [2]:
script_with_logs = """
// Get all elements in the HTML page
const elements = document.getElementsByTagName('*');

// Create an array to store the element details
const elementDetails = [];

// Iterate through each element
for (let i = 0; i < elements.length; i++) {
  const element = elements[i];

  // Get the bounding rectangle of the element
  const rect = element.getBoundingClientRect();

  // Get the text content of the element
  const textContent = element.textContent.trim();

  // Get the tooltip value if it exists
  const tooltip = element.hasAttribute('title') ? element.getAttribute('title') : '';

  // Store the element, its bounding rectangle, text content, and tooltip details
  const elementInfo = {
    element: element,
    rect: rect,
    textContent: textContent,
    ariaLabel: element.hasAttribute('aria-label') ? element.getAttribute('aria-label') : '',
    tooltip: tooltip,
    e_type: element.nodeName
  };
  if (elementInfo.rect.width > 0 && elementInfo.rect.height > 0){
  // Add the element details to the array
   if (elementInfo.tooltip != '' || elementInfo.textContent != '' || elementInfo.ariaLabel != ''){
    elementDetails.push(elementInfo);
    }
  }
}

let parents = elementDetails.map(e=>e.element.parentElement);
let withoutParents = elementDetails.filter(elementDetail=> !parents.includes(elementDetail.element));

// Display the element details
console.log("X,Y,Width,Height,ElementType,textContent,TooltipValue,AriaLabel");
console.log(withoutParents.map( e=> e.rect.x+","+e.rect.y+","+e.rect.width+","+e.rect.height+","+e.e_type+","+e.rect.textContent+","+e.tooltip+","+e.ariaLabel).join("\\n"));
"""

def screen_to_table(wd):
  import pandas as pd
  import io

  script = f"""
  var consoleLogs = [];
  var originalLog = console.log;
  console.log = function(message) {{
      consoleLogs.push(message);
      originalLog.apply(console, arguments);
  }};

  {script_with_logs}

  return consoleLogs;
  """
  logs = wd.execute_script(script)

  df = pd.read_csv(io.StringIO("\n".join(logs)), sep=",")

  return df

In [3]:
def start_browesr():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service

  service = Service(executable_path=r'/usr/bin/chromedriver')
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.headless = True
  return webdriver.Chrome(service=service,options=chrome_options)


def display_image(file_name):
  from IPython.display import Image, display
  display(Image(filename=file_name))



def draw_on_screen(filename,x,y):
  from PIL import Image, ImageDraw
  # Perform mouse click at X and Y coordinates
  # Open the screenshot image using Pillow
  image = Image.open(filename)

  # Create a drawing context on the image
  draw = ImageDraw.Draw(image)

  # Define the size of the marker
  marker_size = 10

  # Draw a marker at the specified coordinates
  draw.rectangle([(x - marker_size, y - marker_size), (x + marker_size, y + marker_size)], outline="red")

  # Save the marked screenshot
  image.save(f"click_location_{filename}")

  after_filename = web_driver_to_image(wd,"after_click_")


# Tools
base on https://github.com/mpaepper/llm_agents/blob/main/llm_agents/tools/google_search.py

In [4]:
from pydantic import BaseModel

class ToolInterface(BaseModel):
    name: str
    description: str
    web_driver: object


    def use(self) -> str:
        raise NotImplementedError("use() method not implemented")  # Implement in subclass

In [5]:
def click_on_screen(wd, x, y):
  from selenium.webdriver.common.action_chains import ActionChains

  actions = ActionChains(wd)
  actions.move_by_offset(x, y)
  actions.click()
  actions.perform()
  return wd


class ClickOnCoordinates(ToolInterface):
  """Click on certain coordinate on the screen """

  name = "Click on coordinates on the screen"
  description = "click on x,y coordinates in order to move to the next screen. Input should be a two value x and y."

  def use(self,web_driver:object, x: float, y:float) -> str:
      return click_on_screen(web_driver,x,y)

In [14]:
class GoToURL(ToolInterface):
  """ Go to a specific url address """

  name = "Go to a specific url web address"
  description = "Change the url to a provied URL. Input should be single string in URL foramt."


  def use(self,web_driver:object, url: str) -> str:
      self.web_driver.get(url)
      return self.web_driver



# AutoAgent



In [7]:
import openai
import os

from pydantic import BaseModel
from typing import List


class ChatLLM(BaseModel):
    model: str = 'gpt-3.5-turbo'
    temperature: float = 0.0
    openai.api_key =

    def generate(self, prompt: str, stop: List[str] = None):
        response = openai.ChatCompletion.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            temperature=self.temperature,
            stop=stop
        )
        return response.choices[0].message.content

In [8]:
def web_driver_to_image(wd,prefix=""):
  import re
  import os
  from urllib.parse import urlparse, urlunparse

  url = wd.current_url
  # Parse the URL
  parsed_url = urlparse(url)

  # Remove the query parameters
  clean_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, "", "", ""))

  print(clean_url)
  file_name = re.sub(r'[^a-zA-Z0-9_-]', '_', clean_url)
  file_name = f"{prefix}_{file_name}.png"

  wd.save_screenshot(file_name)
  return file_name


In [21]:
import datetime
import re

from pydantic import BaseModel
from typing import List, Dict, Tuple


FINAL_ANSWER_TOKEN = "Final Answer:"
OBSERVATION_TOKEN = "Observation:"
THOUGHT_TOKEN = "Thought:"
PROMPT_TEMPLATE = """

Today is {today}, The site i'm looking on is {site_url}, Here is a representation of what is see on my screen in a table shape.

{on_screen_data}

You should accomplish the task given to you as best as you can using the following tools:

{tool_description}

Use the following format:

Question: the input question you must answer
Thought: comment on what you want to do next
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question

Begin!

Task To Accomplish: {task_to_accomplish}
Thought: {previous_responses}
"""


class Agent(BaseModel):
    llm: ChatLLM
    webdriver : object
    tools: List[ToolInterface]
    prompt_template: str = PROMPT_TEMPLATE
    max_loops: int = 1
    # The stop pattern is used, so the LLM does not hallucinate until the end
    stop_pattern: List[str] = [f'\n{OBSERVATION_TOKEN}', f'\n\t{OBSERVATION_TOKEN}']

    @property
    def tool_description(self) -> str:
        return "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])

    @property
    def tool_names(self) -> str:
        return ",".join([tool.name for tool in self.tools])

    @property
    def tool_by_names(self) -> Dict[str, ToolInterface]:
        return {tool.name: tool for tool in self.tools}

    def run(self, task_to_accomplish: str, url:str):
        webdriver = start_browesr()
        try:
            webdriver.get(url)
            previous_responses = []
            num_loops = 0
            on_screen = screen_to_table(self.webdriver)
            web_driver_to_image(self.webdriver,prefix="step_0")
            prompt = self.prompt_template.format(
                    today = datetime.date.today(),
                    tool_description=self.tool_description,
                  tool_names=self.tool_names,
                  task_to_accomplish=task_to_accomplish,
                  on_screen_data="{on_screen_data}",
                  previous_responses='{previous_responses}',
                  site_url=url
            )

            while num_loops < self.max_loops:
                num_loops += 1

                curr_prompt = prompt.format(previous_responses='\n'.join(previous_responses),
                                            on_screen_data=on_screen.to_csv(index=False)
                )
                print(curr_prompt)


                generated, tool, tool_input = self.decide_next_action(curr_prompt)
                if tool == 'Final Answer':
                    return tool_input
                if tool not in self.tool_by_names:
                    raise ValueError(f"Unknown tool: {tool}")

                print('------')
                print(generated)

                self.webdriver = self.tool_by_names[tool].use(self.webdriver,tool_input)
                on_screen = screen_to_table(self.webdriver)
                web_driver_to_image(self.webdriver,prefix=f"step_{str(num_loops+1)}")
                generated += f"\n{OBSERVATION_TOKEN} \n{THOUGHT_TOKEN}"

                previous_responses.append(generated)
        except Exception:
                if webdriver:
                  webdriver.close()


    def decide_next_action(self, prompt: str) -> str:
        generated = self.llm.generate(prompt, stop=self.stop_pattern)

        tool, tool_input = self._parse(generated)
        return generated, tool, tool_input

    def _parse(self, generated: str) -> Tuple[str, str]:
        if FINAL_ANSWER_TOKEN in generated:
            return "Final Answer", generated.split(FINAL_ANSWER_TOKEN)[-1].strip()

        regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, generated, re.DOTALL)
        if not match:
            raise ValueError(f"Output of LLM is not parsable for next tool use: `{generated}`")
        tool = match.group(1).strip()
        tool_input = match.group(2)
        return tool, tool_input.strip(" ").strip('"')

In [22]:

agent = Agent(llm=ChatLLM(), tools=[GoToURL(),ClickOnCoordinates()])
agent.run("Log into my Gmail account","https://www.google.com")

<ipython-input-3-fcf916d6169a>:9: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


KeyboardInterrupt: ignored